In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (6, 2.2)

x = np.linspace(-4, 4, 400)

# define three normal pdfs with different means/variances
def normal_pdf(x, mu, sigma):
    return 1/(np.sqrt(2*np.pi)*sigma) * np.exp(-(x-mu)**2/(2*sigma**2))

prior = normal_pdf(x, -0.5, 1.5)
post1 = normal_pdf(x, 1.0, 0.8)
post2 = normal_pdf(x, 2.0, 0.6)

plt.plot(x, prior, color="gray", label="Prior", linewidth=2)
plt.plot(x, post1, color="orange", label="Posterior 1", linewidth=2)
plt.plot(x, post2, color="red", label="Posterior 2", linewidth=2)

# arrow
# arrow_x = 0.7
# arrow_y = normal_pdf(arrow_x, 1.0, 0.8)
plt.annotate('', xy=(1.2, normal_pdf(1.2, 1.0, 0.8)), 
            #  xytext=(arrow_x, arrow_y),
             arrowprops=dict(arrowstyle="->", color="black"))


plt.tight_layout()



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.style.use("default")          # 先切回默认风格
plt.rcParams["figure.figsize"] = (6, 2.2)

x = np.linspace(-4, 4, 400)

def normal_pdf(x, mu, sigma):
    return 1/(np.sqrt(2*np.pi)*sigma) * np.exp(-(x-mu)**2/(2*sigma**2))

plt.plot(x, normal_pdf(x, -0.5, 1.5), color="orange", lw=2, label="Prior")
plt.plot(x, normal_pdf(x, 1.0, 0.8),  color="#F46920", lw=2,  label="Posterior 1")
plt.plot(x, normal_pdf(x, 2.0, 0.6),  color="red", lw=2, label="Posterior 2")

plt.annotate("", xy=(1.2, normal_pdf(1.2, 1.0, 0.8)),
             arrowprops=dict(arrowstyle="->"))

ax = plt.gca()
ax.tick_params(labelbottom=False, labelleft=False)   # 去刻度文字

# 保证四条框线可见
for s in ax.spines.values():
    s.set_visible(True)
    s.set_color("black")
    s.set_linewidth(1.0)

plt.tight_layout()
plt.savefig("bayesianflow.png", dpi=300, bbox_inches="tight")
plt.show()



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ----------------------------  Parameters  ----------------------------
np.random.seed(0)
n_time  = 800              # horizontal resolution
n_space = 250              # vertical resolution
t_final = 2.0              # rightmost time (for extent)
y_min, y_max = -1.2, 1.0   # vertical axis limits

# -------------------  Build triangular-density "wedge"  -------------------
x = np.linspace(0, 1, n_time)                            # normalized [0,1]
sigma_left  = 0.90 * (y_max - y_min) / 2                 # max half‑width on left
sigma_right = 0.02                                        # almost zero on right

sigma = sigma_left * (1 - x) + sigma_right * x           # linear taper
y_center = 0.0                                            # centerline

# Create a 2‑D array where each column is a Gaussian with its own sigma
Y = np.linspace(y_min, y_max, n_space)[:, None]           # column vector
gauss_cols = np.exp(-0.5 * ((Y - y_center) / sigma)**2)   # shape (n_space, n_time)

# Normalize columns to [0,1] for consistent brightness
gauss_cols /= gauss_cols.max(axis=0)

# ----------------------  Simulated random trajectories  ----------------------
n_traces = 7
t = np.linspace(0, t_final, n_time)

traces = []
for _ in range(n_traces):
    # start somewhere inside the leftmost wedge (within 70% of full width)
    y0 = (np.random.rand() - 0.5) * 0.7 * 2 * sigma_left
    # Brownian increments
    dy = 0.03 * np.random.randn(n_time).cumsum()
    # Linear drift toward 0 so curves meet at the tip
    y = y0 * (1 - x) + dy * (1 - x)          # multiply by (1‑x) keeps narrowing
    traces.append(y_center + y)

# ----------------------------  Plot  ----------------------------
plt.rcParams["figure.figsize"] = (7, 2.0)

fig, ax = plt.subplots()

# Background density map
ax.imshow(
    gauss_cols,
    extent=[0, t_final, y_min, y_max],
    origin="lower",                  # y_min at bottom
    aspect="auto"                    # fill horizontally
)

# Overlay random traces (default color cycle)
for y in traces:
    ax.plot(t, y, lw=1.3)

# Vertical dashed line at t = t_final
ax.axvline(t_final, ls="--", lw=1)

# Remove axes and ticks
ax.set_xlim(0, t_final)
ax.set_ylim(y_min, y_max)
ax.set_xticks([])
ax.set_yticks([])
for spine in ax.spines.values():
    spine.set_visible(False)

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ---------- 基本参数 ----------
np.random.seed(0)
n_time  = 800          # 采样点
t_final = 2.0          # 右端时间
y_min, y_max = -1.2, 1.0
n_traces = 7           # 曲线数量

t = np.linspace(0, t_final, n_time)
x_norm = t / t_final                  # 归一化时间 0→1
sigma_left = 0.9 * (y_max - y_min) / 2

# ---------- 生成曲线 ----------
traces = []
for _ in range(n_traces):
    y0 = (np.random.rand() - 0.5) * 0.7 * 2 * sigma_left     # 左侧起点分散
    step_scale = 0.05 * (1 - x_norm)                         # 步长随 x 缩小
    steps = step_scale * np.random.randn(n_time)
    noise = np.cumsum(steps)                                 # 随机游走
    y = y0 * (1 - x_norm) + noise * (1 - x_norm)             # 线性收敛 + 噪声收敛
    traces.append(y)

# ---------- 绘图 ----------
plt.rcParams["figure.figsize"] = (7, 2.0)
fig, ax = plt.subplots(facecolor="white")

for y in traces:
    ax.plot(t, y, lw=1.3)

ax.set_xlim(0, t_final)
ax.set_ylim(y_min, y_max)
ax.set_xticks([]); ax.set_yticks([])
for spine in ax.spines.values():
    spine.set_visible(False)

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ----------------------- 参数 -----------------------
np.random.seed(0)
n_time  = 800
t_final = 2.0
y_min, y_max = -7, 7          # 更大的纵向范围
n_traces = 7

t = np.linspace(0, t_final, n_time)
x_norm = t / t_final

sigma_left = 0.9 * (y_max - y_min) / 2   # 0.9*(14)/2 = 6.3

# ----------------------- 生成曲线 -----------------------
traces = []
for _ in range(n_traces):
    # 左端随机起点：±2*sigma_left ≈ ±12.6
    y0 = (np.random.rand() - 0.5) * 3 * sigma_left
    
    # 噪声步长放大到 0.25，并随 (1-x)^2 衰减
    step_scale = 0.25 * (1 - x_norm)**2
    steps = step_scale * np.random.randn(n_time)
    noise = np.cumsum(steps)
    
    # 收敛到 y=0
    y = (y0 + noise) * (1 - x_norm)**2
    traces.append(y)

# ----------------------- 绘图 -----------------------
plt.rcParams["figure.figsize"] = (7, 2.2)
fig, ax = plt.subplots(facecolor="white")

for y in traces:
    ax.plot(t, y, lw=1.2)

ax.set_xlim(0, t_final)
ax.set_ylim(y_min, y_max)
ax.set_xticks([]); ax.set_yticks([])
for spine in ax.spines.values():
    spine.set_visible(False)

plt.tight_layout()
plt.savefig("sdesolver.png", dpi=300, bbox_inches="tight")
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ----------------------- 参数 -----------------------
np.random.seed(0)
n_time  = 800
t_final = 2.0             # 原始时间范围
cut_frac = 0.9            # 截断到 90% 处，可自行调
n_traces = 7

# 生成完整时间轴，然后根据 cut_frac 截断
t_full = np.linspace(0, t_final, n_time)
cut_index = int(n_time * cut_frac)
t = t_full[:cut_index+1]              # 保留 0 → t_cut

x_norm = t_full / t_final             # 用于噪声控制
x_norm_cut = x_norm[:cut_index+1]

# 散度参数
y_min, y_max = -7, 7
sigma_left = 0.9 * (y_max - y_min) / 2

# ----------------------- 生成曲线 -----------------------
traces = []
for _ in range(n_traces):
    y0 = (np.random.rand() - 0.5) * 3 * sigma_left
    step_scale_full = 0.25 * (1 - x_norm)**2
    noise_full = np.cumsum(step_scale_full * np.random.randn(n_time))
    # 收敛到 y=0，但我们稍早截断
    y_full = (y0 + noise_full) * (1 - x_norm)**2
    traces.append(y_full[:cut_index+1])

# 自动 y 轴范围（留一点 margin）
y_all = np.concatenate(traces)
y_min_plot, y_max_plot = y_all.min()*1.05, y_all.max()*1.05

# ----------------------- 绘图 -----------------------
plt.rcParams["figure.figsize"] = (7, 2.2)
fig, ax = plt.subplots(facecolor="white")

for y in traces:
    ax.plot(t, y, lw=2)

# 黑色外框
for side in ["top", "right", "bottom", "left"]:
    ax.spines[side].set_visible(True)
    ax.spines[side].set_color("black")
    ax.spines[side].set_linewidth(1.4)

ax.set_xlim(0, t[-1])
ax.set_ylim(y_min_plot, y_max_plot)
ax.set_xticks([]); ax.set_yticks([])

plt.tight_layout()
out_file = "sdesolver.png"
plt.savefig(out_file, dpi=500, bbox_inches="tight")
plt.show()

# out_file
